In [ ]:
from pathlib import Path

from sed import SedProcessor
import sed
import numpy as np


%matplotlib inline
# %matplotlib ipympl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
config_file = Path(sed.__file__).parent.parent/'tutorial/hextof_config.yaml'
assert config_file.exists()

# Loading Data

In [ ]:
config={"core": {"paths": {
    "data_raw_dir": "/asap3/flash/gpfs/pg2/2023/data/11019101/raw/hdf/offline/fl1user3", 
    "data_parquet_dir": "/home/agustsss/temp/sed_parquet/"
}}}
sp = SedProcessor(runs=[44797], config=config, user_config=config_file, system_config={}, collect_metadata=False)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()

# time-of-flight spectrum

In [ ]:
sp.append_tof_ns_axis()

In [ ]:
axes = ['sampleBias','dldTime']
bins = [5, 250]
ranges = [[28,33],  [650,800]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.plot.line(x='dldTime')

# Energy Calibration

## using lmfit

In [ ]:
axes = ['sampleBias', 'dldTimeSteps']
bins = [5, 500]
ranges = [[28,33], [4000, 4800]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
dres = res.copy()
dres.data = np.gradient(res.data, axis=1)
dres.plot.line(x='dldTimeSteps')

In [ ]:
sp.load_bias_series(binned_data=dres)

In [ ]:
ranges=(4120, 4200)
ref_id=0
sp.find_bias_peaks(ranges=ranges, ref_id=ref_id)

In [ ]:
axes = ['sampleBias', 'dldTimeSteps']
bins = [5, 2000]
ranges = [[28,33], [0, 2000]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.plot.line(x='dldTimeSteps')

In [ ]:
# ref_id=0
# ref_energy=0
# sp.calibrate_energy_axis(
#     ref_id=ref_id,
#     ref_energy=ref_energy,
#     method="lmfit",
#     energy_scale='kinetic',
# )

In [ ]:
from sed.calibrator.energy import tof2ev, tof2ns

In [ ]:
ph_peak = 1e-9 * tof2ns(binwidth=sp.config['dataframe']['tof_binwidth'], binning=3,t=219)
sp.calibrate_energy_axis(
    ref_id=0,
    ref_energy=0,
    method="lmfit",
    energy_scale='kinetic',
    d={'value':1.,'min': .8, 'max':5, 'vary':True},
    t0={'value':ph_peak, 'min': ph_peak*0.1, 'max': ph_peak*1.9, 'vary':False},
    E0={'value': 0., 'min': -100, 'max': 100, 'vary': True},

)

In [ ]:
# tof = np.linspace(4000,5000,500)
# plt.figure()
# plt.plot(tof, tof2ev(
#     t=tof, 
#     tof_distance=1, 
#     time_offset=1e-9*tof2ns(binwidth=sp.config['dataframe']['tof_binwidth'], binning=3,t=218), 
#     energy_offset=0,
#     binning=3,
#     energy_scale='kinetic',
#     binwidth=sp.config['dataframe']['tof_binwidth'])
# )

In [ ]:
sp.append_energy_axis()

In [ ]:
sp.ec.calibration

In [ ]:
sp.dataframe[['dldTime','dldTimeSteps','energy','dldSectorID']].head()

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,10]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.mean('sampleBias').plot.line(x='energy',linewidth=3)
res.plot.line(x='energy',linewidth=1,alpha=.5);

In [ ]:
sp.dataframe

In [ ]:
sp.apply_energy_offset(
    constant=2,
    columns=['sampleBias','monochromatorPhotonEnergy','tofVoltage'],
    signs=[1,-1,-1],
    subtract_mean=True,
)

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,2]]
res_fit = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
ax = plt.subplot(111)
res_fit.energy.attrs['unit'] = 'eV'
res_fit.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax)
res_fit.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax);

# Fit the falling edge at E=0 to evaluate the quality of the energy calibration

In [ ]:
from lmfit.models import Gaussian2dModel, LorentzianModel, LinearModel

In [ ]:
curve = res_fit.mean('sampleBias').sel(energy=slice(-0.15,1))
x = curve.energy
y = -np.gradient(curve.data)
full_y = np.gradient(res_fit.data, axis=1)
gm = LorentzianModel()
# lin = LinearModel()
model = gm #+ lin
params = gm.guess(y, x=x)
# params.update(lin.make_params())
result = model.fit(y, params, x=x)

plt.figure()
plt.plot(x, y, 'bo')
plt.plot(x, result.best_fit, 'r-')
result.best_values
best = result.params

In [ ]:
{'amplitude': 62.649384173227375,
 'center': 0.07618539040289204,
 'sigma': 0.0948828689156287}

In [ ]:
plt.figure()
for i in range(len(res_fit.sampleBias)):
    x = res_fit.isel(sampleBias=i).sel(energy=slice(-0.15,1)).energy
    y = res_fit.isel(sampleBias=i).sel(energy=slice(-0.15,1)).data
    y = -np.gradient(y)
    y = y/np.max(y)
    result = model.fit(y, best, x=x)
    print(f'{i}: {result.best_values}')
    plt.plot(x, y+i, 'bo')
    plt.plot(x, result.best_fit+i, 'r-')
